In [5]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [6]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from gensim import corpora, models
import nltk
import feedparser


In [8]:
#주제식별

class IdentifyingTopicExample:
    def getDocuments(self):
        url = 'https://sports.yahoo.com/mlb/rss.xml'
        feed = feedparser.parse(url)
        self.documents = []
        for entry in feed['entries'][:5]:
            text = entry['summary']#요약 
            if 'ex' in text:
                continue
            self.documents.append(text)
            print("-- {}".format(text))
        print("INFO: Fetching documents from {} completed".format(url))

    def cleanDocuments(self):
        tokenizer = RegexpTokenizer(r'[a-zA-Z]+')#각 토큰은 a에서 z, A-Z사이의 문자로 구성
        en_stop = set(stopwords.words('english'))# 영어 불용어 제거
        self.cleaned = []#토큰화, 불용어 된 문서를 저장하는 리스트
        for doc in self.documents:
            lowercase_doc = doc.lower()#문서를 소문자로 변환
            words = tokenizer.tokenize(lowercase_doc)
            non_stopped_words = [i for i in words if not i in en_stop]#문장의 모든 단어가 영어 불용어 카테고리에 속해 있으면 무시하고,
            #나머지는 모두 non_stopped_words 에 저장
            self.cleaned.append(non_stopped_words)
        print("INFO: Clearning {} documents completed".format(len(self.documents)))

    def doLDA(self):#LDA : 각 문서에 담긴 단어들의 주제를 추출하는 토픽 모델링 기법
        dictionary = corpora.Dictionary(self.cleaned)
        corpus = [dictionary.doc2bow(cleandoc) for cleandoc in self.cleaned]#입력된 말뭉치는 정리된 각 문장에 대한 단어들의 모음으로 정의
        ldamodel = models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary)#주제 수를 2로 정한 말뭉치 모델 생성,어휘크기 설정
        print(ldamodel.print_topics(num_topics=2, num_words=4))#각 주제 마다 네 개의 단어를 포함하는 두 개의 주제를 화면에 출력 

    def run(self):
        self.getDocuments()
        self.cleanDocuments()
        self.doLDA()

In [9]:
if __name__ == '__main__':
    topicExample = IdentifyingTopicExample()
    topicExample.run()

-- Here is the latest on the New York Mets' front office search...
-- Second base just might be the White Sox' biggest need this winter. Here are some free agents who could be the solution.
INFO: Fetching documents from https://sports.yahoo.com/mlb/rss.xml completed
INFO: Clearning 2 documents completed
[(0, '0.071*"biggest" + 0.069*"white" + 0.067*"need" + 0.067*"could"'), (1, '0.072*"latest" + 0.072*"search" + 0.072*"front" + 0.072*"york"')]
